***GENERATED CODE FOR keepnokeepxgb PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import warnings
warnings.filterwarnings('ignore')


class RDBMSConnector:

    def fetch(spark, config):
        drivers = {"mssql": "com.microsoft.sqlserver.jdbc.SQLServerDriver"}
        return spark.read.format("jdbc") \
            .option("url", f"jdbc:sqlserver://{eval(config)['host']}:{eval(config)['port']};databaseName={eval(config)['database']}") \
            .option(eval(config)['qtype'], eval(config)['query']) \
            .option("user", eval(config)['user']) \
            .option("password", eval(config)['password']) \
            .option("driver",  "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
            .load()

    def put(df, spark, config):
        drivers = {"mssql": "com.microsoft.sqlserver.jdbc.SQLServerDriver"}
        # Write modes: overwrite, append
        df.write.mode(eval(config)['writemode'])\
            .format('jdbc') \
            .option("url", f"jdbc:{eval(config)['dbtype']}://{eval(config)['host']}:{eval(config)['port']};databaseName={eval(config)['database']}") \
            .option("dbtable", eval(config)['table']) \
            .option("user", eval(config)['user']) \
            .option("password", eval(config)['password']) \
            .option("driver",  "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
            .save()


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
from pyspark.sql.functions import dayofmonth, month, year, col
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


def ExtractDateTransform(df, params, transformationData={}):
    transform_params = params
    dfReturn = df
    feature = transform_params['feature']
    dfReturn = dfReturn.fillna({feature: ''})
    dfReturn = dfReturn.withColumn(
        feature+'dayofmonth', dayofmonth(col(feature)))
    dfReturn = dfReturn.withColumn(feature+'month', month(col(feature)))
    dfReturn = dfReturn.withColumn(feature+'year', year(col(feature)))
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = transformationDF.drop('AppointmentID')
        transformationDF = transformationDF.drop('PatientNumber')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Gender', 'transformation_label': 'String Indexer'}], 'feature': 'Gender', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1167', 'mean': '', 'stddev': '', 'min': 'F', 'max': 'M', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Gender'}, {'feature_label': 'Gender', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Gender')
        transformationDF = transformationDF.drop('PatientLoadDate')
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'AppointmentDate', 'transformation_label': 'Extract Date'}], 'feature': 'AppointmentDate', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
            'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 'AppointmentDate'}, {'feature_label': 'AppointmentDate', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop('AppointmentDate')
        transformationDF = transformationDF.drop('AppointmentCreatedDate')
        transformationDF = transformationDF.drop('ModifiedDate')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'AppointmnetType', 'transformation_label': 'String Indexer'}], 'feature': 'AppointmnetType', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1167', 'mean': '', 'stddev': '', 'min': '*ESTABLISHED PATIENT', 'max': 'THERAPY', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'AppointmnetType'}, {'feature_label': 'AppointmnetType', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('AppointmnetType')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PracticeName', 'transformation_label': 'String Indexer'}], 'feature': 'PracticeName', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1167', 'mean': '', 'stddev': '', 'min': 'MS PG1', 'max': 'MS PG1', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PracticeName'}, {'feature_label': 'PracticeName', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PracticeName')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PracticeSpeciality', 'transformation_label': 'String Indexer'}], 'feature': 'PracticeSpeciality', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1167', 'mean': '', 'stddev': '', 'min': 'Multi-Specialty', 'max': 'Multi-Specialty', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PracticeSpeciality'}, {'feature_label': 'PracticeSpeciality', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PracticeSpeciality')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PracticeRegion', 'transformation_label': 'String Indexer'}], 'feature': 'PracticeRegion', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1167', 'mean': '', 'stddev': '', 'min': 'SouthEast', 'max': 'SouthEast', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PracticeRegion'}, {'feature_label': 'PracticeRegion', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PracticeRegion')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'FacilityCategory', 'transformation_label': 'String Indexer'}], 'feature': 'FacilityCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1167', 'mean': '', 'stddev': '', 'min': 'SocCLEXX', 'max': 'SocUniXX', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'FacilityCategory'}, {'feature_label': 'FacilityCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('FacilityCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'FacilityCity', 'transformation_label': 'String Indexer'}], 'feature': 'FacilityCity', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1167', 'mean': '', 'stddev': '', 'min': 'HOUSTON', 'max': 'Sugar Land', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'FacilityCity'}, {'feature_label': 'FacilityCity', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('FacilityCity')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'FacilityState', 'transformation_label': 'String Indexer'}], 'feature': 'FacilityState', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1167', 'mean': '', 'stddev': '', 'min': 'TX', 'max': 'TX', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'FacilityState'}, {'feature_label': 'FacilityState', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('FacilityState')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'FacilityZip', 'transformation_label': 'String Indexer'}], 'feature': 'FacilityZip', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1167', 'mean': '771714708.7', 'stddev': '1775332.75', 'min': '770025351', 'max': '775053948', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'FacilityZip'}, {'feature_label': 'FacilityZip', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('FacilityZip')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ProviderName', 'transformation_label': 'String Indexer'}], 'feature': 'ProviderName', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1167', 'mean': '', 'stddev': '', 'min': 'NoelAhmyyy, HassaXXX', 'max': 'NoelYooyyy, JohnXXX', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ProviderName'}, {'feature_label': 'ProviderName', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ProviderName')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Speciality', 'transformation_label': 'String Indexer'}], 'feature': 'Speciality', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1167', 'mean': '', 'stddev': '', 'min': 'Allergy/Immun', 'max': 'Speech Pathologists', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Speciality'}, {'feature_label': 'Speciality', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Speciality')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ProviderTaxonomyName', 'transformation_label': 'String Indexer'}], 'feature': 'ProviderTaxonomyName', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1167', 'mean': '', 'stddev': '', 'min': 'Allopathic & Osteopathic Physicians : Allergy & Immunology', 'max': 'Unspecified', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ProviderTaxonomyName'}, {'feature_label': 'ProviderTaxonomyName', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ProviderTaxonomyName')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ProviderGroup1', 'transformation_label': 'String Indexer'}], 'feature': 'ProviderGroup1', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1167', 'mean': '', 'stddev': '', 'min': 'Allergy', 'max': 'Unspecified', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ProviderGroup1'}, {'feature_label': 'ProviderGroup1', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ProviderGroup1')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ProviderGroup2', 'transformation_label': 'String Indexer'}], 'feature': 'ProviderGroup2', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1167', 'mean': '', 'stddev': '', 'min': 'Allergy', 'max': 'Unspecified', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ProviderGroup2'}, {'feature_label': 'ProviderGroup2', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ProviderGroup2')
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'ProviderCreatedDate', 'transformation_label': 'Extract Date'}], 'feature': 'ProviderCreatedDate', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
            'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 'ProviderCreatedDate'}, {'feature_label': 'ProviderCreatedDate', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop('ProviderCreatedDate')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ReferringProviderName', 'transformation_label': 'String Indexer'}], 'feature': 'ReferringProviderName', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1167', 'mean': '', 'stddev': '', 'min': 'Abxxxael', 'max': 'Zoxxxyum', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ReferringProviderName'}, {'feature_label': 'ReferringProviderName', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ReferringProviderName')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PayerID', 'transformation_label': 'String Indexer'}], 'feature': 'PayerID', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1167', 'mean': '', 'stddev': '', 'min': '011FD2FB-E83C-4B55-BA44-9F0ACAE80BD6', 'max': 'F4F43E5D-FBE7-4FC0-BC0C-B3CEEA59CB6E', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PayerID'}, {'feature_label': 'PayerID', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PayerID')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PayerCategory', 'transformation_label': 'String Indexer'}], 'feature': 'PayerCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1167', 'mean': '', 'stddev': '', 'min': 'Aetna', 'max': 'United Healthcare', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PayerCategory'}, {'feature_label': 'PayerCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PayerCategory')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from sklearn.model_selection import train_test_split
import lightgbm as lgb


def gradientboostingclassifier(df, labels, features):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    lightgbm_clf = lgb.LGBMClassifier()
    X_train, X_test, y_train, y_test = train_test_split(
        df[features], df[labels])
    clf.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % lightgbm_clf.score(X_test, y_test))
    data = {'model': lightgbm_clf,
            'X_test': X_test,
            'y_test': y_test,
            'label': labels,
            'columnNames': df.columns}
    return data


***READING DATAFRAME***

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

#%run keepnokeepxgbHooks.ipynb
try:
	#sourcePreExecutionHook()

	sqlserver = RDBMSConnector.fetch(spark, "{'host': 'wsc-sqlserver1.database.windows.net', 'port': '1433', 'password': 'G5gTW5rnKVeAZWXm', 'database': 'IntelligentScheduling', 'optionalDB': 'IntelligentScheduling', 'user': 'Numtra', 'qtype': 'query', 'query': "SELECT * FROM\r\n\t\t     (\r\n\t\t     \r\n\t\t     \t\t\tSELECT DISTINCT\r\n\t\t     \t\t\tA.AppointmentID,\r\n\t\t     \t\t\tP.PatientNumber, P.Gender, P.LoadDate as PatientLoadDate,\r\n\t\t     \t\t\tFLOOR(DATEDIFF(DAY, P.DateOfBirth, GETDATE()) / 365) AS Age,\r\n\t\t     \t\t\tA.AppointmentDate,A.CreatedDate as AppointmentCreatedDate,\r\n\t\t     \t\t\tA.ModifiedDate,\r\n\t\t     \t\t\tA.AppointmentDuration,\r\n\t\t     \t\t\tAT.AppointmentTypeCategory as AppointmnetType,\r\n\t\t     \t\t\tA.EligibilityFlag, A.ConfirmationFlag, \r\n\t\t     \t\t\tPRC.PracticeName, PRC.Speciality as PracticeSpeciality, PRC.Region as PracticeRegion,\r\n\t\t     \t\t\tF.FacilityCategory, F.City as FacilityCity,F.State as FacilityState,F.Zip as FacilityZip,\r\n\t\t     \t\t\tPR.ProviderCategory as ProviderName, SP.SpecialityCategory as Speciality, PTx.ProviderTaxonomyCategory as ProviderTaxonomyName,\r\n\t\t     \t\t\tPR.ProviderGroup1, PR.ProviderGroup2, PR.LoadDate as ProviderCreatedDate,\r\n\t\t     \t\t\tRP.ReferringProviderCategory as ReferringProviderName,\r\n\t\t     \t\t\tPY.PayerID,\r\n\t\t     \t\t\tPY.PayerCategory,\r\n\t\t     \t\t\tLabel=1\r\n\t\t     \r\n\t\t     \t\t\tFROM APPOINTMENT A\r\n\t\t     \t\t\tLEFT OUTER JOIN  APPOINTMENTSTATUS  S on S.AppointmentStatusID=A.AppointmentStatusID\r\n\t\t     \t\t\tLEFT OUTER JOIN APPOINTMENTTYPE AT ON AT.AppointmentTypeID=A.AppointmentTypeID\r\n\t\t     \t\t\tLEFT OUTER JOIN ENCOUNTER EN ON EN.EncounterID=A.EncounterID\r\n\t\t     \t\t\tLEFT OUTER JOIN ENCOUNTERSTATUS ES ON ES.EncounterStatusID=EN.EncounterStatusID\r\n\t\t     \t\t\tLEFT OUTER JOIN PATIENT P ON P.PatientID=A.PatientID\r\n\t\t     \t\t\tLEFT OUTER JOIN PROVIDER PR ON PR.ProviderID=A.ProviderID\r\n\t\t     \t\t\tLEFT OUTER JOIN ProviderTaxonomy PTx ON PR.TaxonomyID=PTx.ProviderTaxonomyID\r\n\t\t     \t\t\tLEFT OUTER JOIN ReferringProvider RP ON RP.ReferringProviderID=A.ReferringProviderID\r\n\t\t     \t\t\tLEFT OUTER JOIN SPECIALITY SP ON SP.SpecialityID=PR.SpecialityID\r\n\t\t     \t\t\tLEFT OUTER JOIN FACILITY F ON F.Facilityid=A.FacilityID\r\n\t\t     \t\t\tLEFT OUTER JOIN PRACTICE PRC ON PRC.PracticeID=A.PracticeID\r\n\t\t     \t\t\tLEFT OUTER JOIN ChargeDetail  CD on CD.PatientID=A.PatientID\r\n\t\t     \t\t\tLEFT OUTER JOIN PAYER PY ON PY.PayerID=CD.PayerID\r\n\t\t     \t\t\tWHERE\r\n\t\t     \t\t\tA.AppointmentDate between  '12/01/2020' and '12/31/2020'\r\n\t\t     \t\t\tAND (\r\n\t\t\t\t\t\t(A.RescheduleFlag = 'Y')\r\n\t\t\t\t\t\tOR (A.RescheduleFlag = 'N'\r\n\t     \t\t\tAND S.AppointmentStatusCategory  =  'Expected'\r\n\t     \t\t\tAND A.DeleteFlag = 'N'\r\n\t     \t\t\tAND  A.AppointmentDate < Getdate())\r\n\t\t\t\t\tOR (A.DeleteFlag = 'Y' OR S.AppointmentStatusCategory  =  'Cancel')\r\n\t\t     \t\t)\t\r\n\t\t     UNION\r\n\t\t     \r\n\t\t     SELECT DISTINCT\r\n\t\t     \r\n\t     \t\t\tA.AppointmentID,\r\n\t     \t\t\tP.PatientNumber, P.Gender, P.LoadDate as PatientLoadDate,\r\n\t     \t\t\tFLOOR(DATEDIFF(DAY, P.DateOfBirth, GETDATE()) / 365) AS Age,\r\n\t     \t\t\tA.AppointmentDate,A.CreatedDate as AppointmentCreatedDate,\r\n\t     \t\t\tA.ModifiedDate,\r\n\t     \t\t\tA.AppointmentDuration,\r\n\t     \t\t\tAT.AppointmentTypeCategory as AppointmnetType,\r\n\t     \t\t\tA.EligibilityFlag, A.ConfirmationFlag, \r\n\t     \t\t\tPRC.PracticeName, PRC.Speciality as PracticeSpeciality, PRC.Region as PracticeRegion,\r\n\t     \t\t\tF.FacilityCategory, F.City as FacilityCity,F.State as FacilityState,F.Zip as FacilityZip,\r\n\t     \t\t\tPR.ProviderCategory as ProviderName, SP.SpecialityCategory as Speciality, PTx.ProviderTaxonomyCategory as ProviderTaxonomyName,\r\n\t     \t\t\tPR.ProviderGroup1, PR.ProviderGroup2, PR.LoadDate as ProviderCreatedDate,\r\n\t     \t\t\tRP.ReferringProviderCategory as ReferringProviderName,\r\n\t     \t\t\tPY.PayerID,\r\n\t     \t\t\tPY.PayerCategory,\r\n\t     \t\t\tLabel=0\r\n\t     \r\n\t     \t\t\tFROM APPOINTMENT A\r\n\t     \t\t\tLEFT OUTER JOIN  APPOINTMENTSTATUS  S on S.AppointmentStatusID=A.AppointmentStatusID\r\n\t     \t\t\tLEFT OUTER JOIN APPOINTMENTTYPE AT ON AT.AppointmentTypeID=A.AppointmentTypeID\r\n\t     \t\t\tLEFT OUTER JOIN ENCOUNTER EN ON EN.EncounterID=A.EncounterID\r\n\t     \t\t\tLEFT OUTER JOIN ENCOUNTERSTATUS ES ON ES.EncounterStatusID=EN.EncounterStatusID\r\n\t     \t\t\tLEFT OUTER JOIN PATIENT P ON P.PatientID=A.PatientID\r\n\t     \t\t\tLEFT OUTER JOIN PROVIDER PR ON PR.ProviderID=A.ProviderID\r\n\t     \t\t\tLEFT OUTER JOIN ProviderTaxonomy PTx ON PR.TaxonomyID=PTx.ProviderTaxonomyID\r\n\t     \t\t\tLEFT OUTER JOIN ReferringProvider RP ON RP.ReferringProviderID=A.ReferringProviderID\r\n\t     \t\t\tLEFT OUTER JOIN SPECIALITY SP ON SP.SpecialityID=PR.SpecialityID\r\n\t     \t\t\tLEFT OUTER JOIN FACILITY F ON F.Facilityid=A.FacilityID\r\n\t     \t\t\tLEFT OUTER JOIN PRACTICE PRC ON PRC.PracticeID=A.PracticeID\r\n\t     \t\t\tLEFT OUTER JOIN ChargeDetail  CD on CD.PatientID=A.PatientID\r\n\t     \t\t\tLEFT OUTER JOIN PAYER PY ON PY.PayerID=CD.PayerID\r\n\t     \t\t\tWHERE\r\n\t     \t\t\tA.AppointmentDate between  '12/01/2020' and '12/31/2020'\r\n\t\t     \t\t\tAND NOT (\r\n\t\t\t\t\t\t(A.RescheduleFlag = 'Y')\r\n\t\t\t\t\t\tOR (A.RescheduleFlag = 'N'\r\n\t     \t\t\tAND S.AppointmentStatusCategory  =  'Expected'\r\n\t     \t\t\tAND A.DeleteFlag = 'N'\r\n\t     \t\t\tAND  A.AppointmentDate < Getdate())\r\n\t\t\t\t\tOR (A.DeleteFlag = 'Y' OR S.AppointmentStatusCategory  =  'Cancel')\r\n\t\t     \t\t)\t\t     \t\t\t\t\t\t\t\t\t\r\n\t\r\n\t\t\r\n\t     ) as ACD", 'dbtype': 'mssql', 'is_header': 'Use Header Line', 'server_url': '/numtraPlatform/NumtraPlatformV2/uploads/platform/'}")
	#sourcePostExecutionHook(sqlserver)

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run keepnokeepxgbHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(sqlserver,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "AppointmentID", "transformation_label": "String Indexer"}], "feature": "AppointmentID", "type": "string", "selected": "False", "replaceby": "max", "stats": {"count": "1167", "mean": "", "stddev": "", "min": "002C56DD-14AC-40B9-A734-2F5C3E2A49DC", "max": "FFCDDA89-99E5-4857-B9D5-355B27740623", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "AppointmentID"}, {"transformationsData": [{"feature_label": "PatientNumber", "transformation_label": "String Indexer"}], "feature": "PatientNumber", "type": "string", "selected": "False", "replaceby": "max", "stats": {"count": "1167", "mean": "381939.73", "stddev": "193735.45", "min": "        2532", "max": "      603202", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PatientNumber"}, {"transformationsData": [{"feature_label": "Gender", "transformation_label": "String Indexer"}], "feature": "Gender", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1167", "mean": "", "stddev": "", "min": "F", "max": "M", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Gender"}, {"transformationsData": [{"feature_label": "PatientLoadDate", "transformation_label": "Extract Date"}], "feature": "PatientLoadDate", "type": "date", "selected": "False", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "updatedLabel": "PatientLoadDate"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Age", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "1167", "mean": "58.25", "stddev": "19.88", "min": "0", "max": "96", "missing": "0"}, "updatedLabel": "Age"}, {"transformationsData": [{"feature_label": "AppointmentDate", "transformation_label": "Extract Date"}], "feature": "AppointmentDate", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "AppointmentDate"}, {"transformationsData": [{"feature_label": "AppointmentCreatedDate", "transformation_label": "Extract Date"}], "feature": "AppointmentCreatedDate", "type": "date", "selected": "False", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "updatedLabel": "AppointmentCreatedDate"}, {"transformationsData": [{"feature_label": "ModifiedDate", "transformation_label": "Extract Date"}], "feature": "ModifiedDate", "type": "date", "selected": "False", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "updatedLabel": "ModifiedDate"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "AppointmentDuration", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "1167", "mean": "17.06", "stddev": "29.81", "min": "5", "max": "495", "missing": "0"}, "updatedLabel": "AppointmentDuration"}, {"transformationsData": [{"feature_label": "AppointmnetType", "transformation_label": "String Indexer"}], "feature": "AppointmnetType", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1167", "mean": "", "stddev": "", "min": "*ESTABLISHED PATIENT", "max": "THERAPY", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "AppointmnetType"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "EligibilityFlag", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "1167", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "EligibilityFlag"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "ConfirmationFlag", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "1167", "mean": "0.49", "stddev": "0.5", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "ConfirmationFlag"}, {"transformationsData": [{"feature_label": "PracticeName", "transformation_label": "String Indexer"}], "feature": "PracticeName", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1167", "mean": "", "stddev": "", "min": "MS PG1", "max": "MS PG1", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PracticeName"}, {"transformationsData": [{"feature_label": "PracticeSpeciality", "transformation_label": "String Indexer"}], "feature": "PracticeSpeciality", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1167", "mean": "", "stddev": "", "min": "Multi-Specialty", "max": "Multi-Specialty", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PracticeSpeciality"}, {"transformationsData": [{"feature_label": "PracticeRegion", "transformation_label": "String Indexer"}], "feature": "PracticeRegion", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1167", "mean": "", "stddev": "", "min": "SouthEast", "max": "SouthEast", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PracticeRegion"}, {"transformationsData": [{"feature_label": "FacilityCategory", "transformation_label": "String Indexer"}], "feature": "FacilityCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1167", "mean": "", "stddev": "", "min": "SocCLEXX", "max": "SocUniXX", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "FacilityCategory"}, {"transformationsData": [{"feature_label": "FacilityCity", "transformation_label": "String Indexer"}], "feature": "FacilityCity", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1167", "mean": "", "stddev": "", "min": "HOUSTON", "max": "Sugar Land", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "FacilityCity"}, {"transformationsData": [{"feature_label": "FacilityState", "transformation_label": "String Indexer"}], "feature": "FacilityState", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1167", "mean": "", "stddev": "", "min": "TX", "max": "TX", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "FacilityState"}, {"transformationsData": [{"feature_label": "FacilityZip", "transformation_label": "String Indexer"}], "feature": "FacilityZip", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1167", "mean": "771714708.7", "stddev": "1775332.75", "min": "770025351", "max": "775053948", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "FacilityZip"}, {"transformationsData": [{"feature_label": "ProviderName", "transformation_label": "String Indexer"}], "feature": "ProviderName", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1167", "mean": "", "stddev": "", "min": "NoelAhmyyy, HassaXXX", "max": "NoelYooyyy, JohnXXX", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ProviderName"}, {"transformationsData": [{"feature_label": "Speciality", "transformation_label": "String Indexer"}], "feature": "Speciality", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1167", "mean": "", "stddev": "", "min": "Allergy/Immun", "max": "Speech Pathologists", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Speciality"}, {"transformationsData": [{"feature_label": "ProviderTaxonomyName", "transformation_label": "String Indexer"}], "feature": "ProviderTaxonomyName", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1167", "mean": "", "stddev": "", "min": "Allopathic & Osteopathic Physicians : Allergy & Immunology", "max": "Unspecified", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ProviderTaxonomyName"}, {"transformationsData": [{"feature_label": "ProviderGroup1", "transformation_label": "String Indexer"}], "feature": "ProviderGroup1", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1167", "mean": "", "stddev": "", "min": "Allergy", "max": "Unspecified", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ProviderGroup1"}, {"transformationsData": [{"feature_label": "ProviderGroup2", "transformation_label": "String Indexer"}], "feature": "ProviderGroup2", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1167", "mean": "", "stddev": "", "min": "Allergy", "max": "Unspecified", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ProviderGroup2"}, {"transformationsData": [{"feature_label": "ProviderCreatedDate", "transformation_label": "Extract Date"}], "feature": "ProviderCreatedDate", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "ProviderCreatedDate"}, {"transformationsData": [{"feature_label": "ReferringProviderName", "transformation_label": "String Indexer"}], "feature": "ReferringProviderName", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1167", "mean": "", "stddev": "", "min": "Abxxxael", "max": "Zoxxxyum", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ReferringProviderName"}, {"transformationsData": [{"feature_label": "PayerID", "transformation_label": "String Indexer"}], "feature": "PayerID", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1167", "mean": "", "stddev": "", "min": "011FD2FB-E83C-4B55-BA44-9F0ACAE80BD6", "max": "F4F43E5D-FBE7-4FC0-BC0C-B3CEEA59CB6E", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PayerID"}, {"transformationsData": [{"feature_label": "PayerCategory", "transformation_label": "String Indexer"}], "feature": "PayerCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1167", "mean": "", "stddev": "", "min": "Aetna", "max": "United Healthcare", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PayerCategory"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Label", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "1167", "mean": "0.32", "stddev": "0.47", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Label"}, {"feature": "Gender_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "1167", "mean": "0.42", "stddev": "0.49", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Gender_stringindexer_tran..."}, {"feature": "AppointmentDate_dayofmonth", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "1167", "mean": "14.48", "stddev": "9.11", "min": "1", "max": "31", "missing": "0"}, "updatedLabel": "AppointmentDate_dayofmont..."}, {"feature": "AppointmentDate_month", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "1167", "mean": "12.0", "stddev": "0.0", "min": "12", "max": "12", "missing": "0"}, "updatedLabel": "AppointmentDate_month"}, {"feature": "AppointmentDate_year", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "1167", "mean": "2020.0", "stddev": "0.0", "min": "2020", "max": "2020", "missing": "0"}, "updatedLabel": "AppointmentDate_year"}, {"feature": "AppointmnetType_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "1167", "mean": "2.78", "stddev": "4.41", "min": "0.0", "max": "29.0", "missing": "0"}, "updatedLabel": "AppointmnetType_stringind..."}, {"feature": "PracticeName_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "1167", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "PracticeName_stringindexe..."}, {"feature": "PracticeSpeciality_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "1167", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "PracticeSpeciality_string..."}, {"feature": "PracticeRegion_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "1167", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "PracticeRegion_stringinde..."}, {"feature": "FacilityCategory_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "1167", "mean": "2.78", "stddev": "2.46", "min": "0.0", "max": "15.0", "missing": "0"}, "updatedLabel": "FacilityCategory_stringin..."}, {"feature": "FacilityCity_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "1167", "mean": "1.11", "stddev": "1.06", "min": "0.0", "max": "6.0", "missing": "0"}, "updatedLabel": "FacilityCity_stringindexe..."}, {"feature": "FacilityState_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "1167", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "FacilityState_stringindex..."}, {"feature": "FacilityZip_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "1167", "mean": "2.96", "stddev": "2.85", "min": "0.0", "max": "20.0", "missing": "0"}, "updatedLabel": "FacilityZip_stringindexer..."}, {"feature": "ProviderName_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "1167", "mean": "10.5", "stddev": "9.39", "min": "0.0", "max": "40.0", "missing": "0"}, "updatedLabel": "ProviderName_stringindexe..."}, {"feature": "Speciality_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "1167", "mean": "0.49", "stddev": "0.75", "min": "0.0", "max": "4.0", "missing": "0"}, "updatedLabel": "Speciality_stringindexer_..."}, {"feature": "ProviderTaxonomyName_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "1167", "mean": "1.42", "stddev": "1.83", "min": "0.0", "max": "10.0", "missing": "0"}, "updatedLabel": "ProviderTaxonomyName_stri..."}, {"feature": "ProviderGroup1_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "1167", "mean": "0.49", "stddev": "0.75", "min": "0.0", "max": "4.0", "missing": "0"}, "updatedLabel": "ProviderGroup1_stringinde..."}, {"feature": "ProviderGroup2_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "1167", "mean": "0.29", "stddev": "0.68", "min": "0.0", "max": "4.0", "missing": "0"}, "updatedLabel": "ProviderGroup2_stringinde..."}, {"feature": "ProviderCreatedDate_dayofmonth", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "1167", "mean": "21.91", "stddev": "0.65", "min": "17", "max": "22", "missing": "0"}, "updatedLabel": "ProviderCreatedDate_dayof..."}, {"feature": "ProviderCreatedDate_month", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "1167", "mean": "5.02", "stddev": "0.13", "min": "5", "max": "6", "missing": "0"}, "updatedLabel": "ProviderCreatedDate_month"}, {"feature": "ProviderCreatedDate_year", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "1167", "mean": "2020.0", "stddev": "0.0", "min": "2020", "max": "2020", "missing": "0"}, "updatedLabel": "ProviderCreatedDate_year"}, {"feature": "ReferringProviderName_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "1167", "mean": "109.0", "stddev": "127.41", "min": "0.0", "max": "449.0", "missing": "0"}, "updatedLabel": "ReferringProviderName_str..."}, {"feature": "PayerID_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "1167", "mean": "5.2", "stddev": "7.39", "min": "0.0", "max": "51.0", "missing": "0"}, "updatedLabel": "PayerID_stringindexer_tra..."}, {"feature": "PayerCategory_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "1167", "mean": "3.05", "stddev": "2.49", "min": "0.0", "max": "11.0", "missing": "0"}, "updatedLabel": "PayerCategory_stringindex..."}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run keepnokeepxgbHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=LightGBMClassifier(autofe, ["Gender_stringindexer", "Age", "AppointmentDuration", "AppointmnetType_stringindexer", "EligibilityFlag", "ConfirmationFlag", "PracticeName_stringindexer", "PracticeSpeciality_stringindexer", "PracticeRegion_stringindexer", "FacilityCategory_stringindexer", "FacilityCity_stringindexer", "FacilityState_stringindexer", "FacilityZip_stringindexer", "ProviderName_stringindexer", "Speciality_stringindexer", "ProviderTaxonomyName_stringindexer", "ProviderGroup1_stringindexer", "ProviderGroup2_stringindexer", "ReferringProviderName_stringindexer", "PayerID_stringindexer", "PayerCategory_stringindexer", "AppointmentDate_dayofmonth", "AppointmentDate_month", "AppointmentDate_year", "ProviderCreatedDate_dayofmonth", "ProviderCreatedDate_month", "ProviderCreatedDate_year"], "Label")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
